In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import re
import datetime
import warnings
warnings.filterwarnings('ignore')

In [10]:
headers = { 'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.121 Safari/537.36' } 

header = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64;     x64; rv:66.0) Gecko/20100101 Firefox/66.0", "Accept-Encoding":"gzip, deflate",     "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8", "DNT":"1","Connection":"close", "Upgrade-Insecure-Requests":"1"}

In [18]:
ind_list = ['mamaearth','godrej','titan watch','frooti','amul',
           'patanjai','dettol','cinthol','britannia biscuits','streax',
           'himalaya products','society tea','tata tea','fastrack watches','mysore sandal']

In [19]:
pasin = []
reviews=[]
dates = []
stars = []
name = []
helpful = []

In [20]:
for prod in ind_list:
    req = requests.get('https://www.amazon.in/s?k='+prod,headers=headers)
    soup = BeautifulSoup(req.content)
    
    asin = []

    for i in soup.findAll('div', attrs={'class':['sg-col-4-of-24 sg-col-4-of-12 sg-col-4-of-36 s-result-item s-asin sg-col-4-of-28 sg-col-4-of-16 sg-col sg-col-4-of-20 sg-col-4-of-32']}):
        asin.append(i['data-asin'])
    
    asin = asin[1:11]
    
    link=[]
    
    for i in range(0,len(asin)):
        url='https://www.amazon.in/dp/'+asin[i]
        page=requests.get(url,headers=headers)
        soup=BeautifulSoup(page.content)
    
        for i in soup.findAll('a',{'class':['a-link-emphasis a-text-bold']}):
            link.append(i['href'])
    
    link = link[1:11]
    

# each page has 10
    for i in range(0,len(link)):
    
        for j in range(0,2):
        
            url='https://www.amazon.in'+link[i]+'&pageNumber='+str(j)
            rev_page=requests.get(url,headers=headers)
        
            soup=BeautifulSoup(rev_page.content)
        
            for k in soup.findAll('span',{'data-hook':'review-body'}):
                reviews.append(k.text.strip())
            
            for k in soup.findAll('i',{'data-hook':'review-star-rating'}):
                stars.append(k.text.split(' ')[0].split('.')[0])
        
            for k in soup.findAll('span',{'data-hook':'review-date'}):
                dates.append(k.text)
                name.append(link[i].split('/')[1])
                pasin.append(asin[i])
                
            
    

In [21]:
len(pasin) , len(dates) , len(reviews) , len(stars) , len(name) 

(2782, 2782, 2782, 2782, 2782)

In [22]:
rev_dict = {'asin':pasin,'name':name,'date':dates,'rating':stars,'review':reviews}

In [23]:
reviews=pd.DataFrame(data=rev_dict,columns=['asin','name','date','rating','review']) 
reviews.head()

,asin,name,date,rating,review
0,B07W7CTLD1,Mamaearth-Onion-Growth-Control-Redensyl,Reviewed in India on 6 September 2019,1,I bought this hair oil after viewing so many g...
1,B07W7CTLD1,Mamaearth-Onion-Growth-Control-Redensyl,Reviewed in India on 14 August 2019,5,Used This Mama Earth Newly Launched Onion Oil ...
2,B07W7CTLD1,Mamaearth-Onion-Growth-Control-Redensyl,Reviewed in India on 19 October 2019,1,So bad product...My hair falling increase too ...
3,B07W7CTLD1,Mamaearth-Onion-Growth-Control-Redensyl,Reviewed in India on 16 September 2019,1,Product just smells similar to navarathna hair...
4,B07W7CTLD1,Mamaearth-Onion-Growth-Control-Redensyl,Reviewed in India on 18 August 2019,5,I have been trying different onion oil for my ...


In [24]:
reviews.rating = reviews.rating.astype('int')

In [25]:
for i in range(0,len(reviews)):
    reviews.date[i] = re.sub('Reviewed in India on','',reviews.date[i])
    reviews.date[i]= reviews.date[i].strip()
    reviews.date[i] = datetime.datetime.strptime(reviews.date[i],'%d %B %Y').date()

In [28]:
reviews.to_csv('amazon_vfl_reviews.csv',index=False)